# Supplemental scripts for the identification of MADA motif

To use each supplemental script, Click the Run button to the left of the code.

The output of below codes will be accessed from the left folder menu.

**!!! Please run the below code before using scripts in this notebook !!!**

In [ ]:
!pip install biopython
from Bio import SeqIO
import pandas as pd

# Supplemental script 1. Extract NB-ARC sequences based on ZAR1

The below code generate alignment fasta of NB-ARC regions of AtZAR1 ("Alignment_NB_AtZAR1.fasta").

- Sequences that lack the intact p-loop motif (GorAXXXXGKTorS) required for NLR protein function are discarded.

- The sequence gaps in the alignments are deleted.

In [ ]:
#@title Upload alignment protein sequences of NLRs that contains "AtZAR1" sequence
from google.colab import files
files = files.upload()
file_name = list(files.keys())[0]

In [ ]:
# convert sequence to table
IDs = []
seqs = []
for record in SeqIO.parse(file_name, 'fasta'):
    IDs.append(str(record.id))
    seqs.append(list(str(record.seq)))
seqs = pd.DataFrame(seqs, index=IDs)

# extract positions of NBARC of AtZAR1
seqs = seqs.loc[:, seqs.loc["AtZAR1", :] != "-"]
AtZAR1_seq = "".join(seqs.loc["AtZAR1", :].values)
NB_start = AtZAR1_seq.find("NGTDR")
NB_end = AtZAR1_seq.find("LNCRH")

# extract sequences in NBARC positions
seqs = seqs.iloc[:, NB_start:NB_end+5]

# NBARC sequeces contains intact p-loop to fasta
with open("NLR_set_alignment_NBARC_RemGap.fasta", mode='w') as f:
    for i in range(seqs.shape[0]):
        seq = "".join(seqs.iloc[i, :].tolist())
        if (seq[44] == "G") or (seq[44] == "A"):
            if (seq[49:51] == "GK"):
                if (seq[51] == "T") or (seq[51] == "S"):
                  f.write(">"+seqs.index[i])
                  f.write("\n")
                  f.write(seq)
                  f.write("\n")

# Supplemental script 2. Extract sequences of your interest ids from fasta file

The below code extract protein sequences of your interest from a big fasta file.

ex) After you get protein ids of CC-NLRs from phylogenetic tree

ex) After you get protein ids of specific Tribe from MCL analysis.

Required Input file
- fasta file that contains protein sequences
- text file of protein ids that you want to extract (1id written in 1line) like↓

```
Solyc08g075640.2.1
NbS00002061g0001.1
HORVU.MOREX.r3.3HG0295670.1
HORVU.MOREX.r3.7HG0634540.1
LOC_Os10g07400.1
…
```


In [ ]:
#@title Upload protein sequences that contains your interest protein ids
from google.colab import files
files = files.upload()
fasta_file = list(files.keys())[0]

In [ ]:
#@title Upload text file of your interest protein ids
from google.colab import files
files = files.upload()
id_file = list(files.keys())[0]

In [ ]:
# @title Output fasta file name

output_name = "test.fasta" # @param {type:"string"}

In [ ]:
with open(id_file, mode='r') as f:
    ids = f.read()

with open(output_name, mode='w') as f:
    for record in SeqIO.parse(fasta_file, 'fasta'):
        if str(record.id) in ids.split("\n"):
            f.write(">"+str(record.id))
            f.write("\n")
            f.write(str(record.seq))
            f.write("\n")

# Supplemental script 3. Extract N-terminal sequences from alignment NLR sequences

The below code generate fasta file of sequences prior to the NB-ARC domain ("N-terminal.fasta").

N-terminal regions are defined based on the NB-ARC region of Arabidopsis ZAR1.

In [ ]:
#@title Upload alignment protein sequences of NLRs that contains "AtZAR1" sequence
from google.colab import files
files = files.upload()
file_name = list(files.keys())[0]

In [ ]:
# convert sequence to table
IDs = []
seqs = []
for record in SeqIO.parse(file_name, 'fasta'):
    IDs.append(str(record.id))
    seqs.append(list(str(record.seq)))
seqs = pd.DataFrame(seqs, index=IDs)

# identify index of start position of NBARC region
start_AA = "NGTDR"
start = []
start_idx = []
for idx, aa in enumerate(seqs.loc["AtZAR1", :].values):
    if aa == "-":
        pass
    else:
        if len(start) > 0:
            if aa == start_AA[len(start)]:
                start.append(aa)
                start_idx.append(idx)
                if len(start) == 5:
                    break
            else:
                if aa == "N":
                    start = ["N"]
                    start_idx = [idx]
                else:
                    start = []
                    start_idx = []
        elif aa == "N":
            start = [aa]
            start_idx = [idx]

# extract N-terminal sequences
with open("N-terminal.fasta", mode='w') as f:
    tmp_seqs = seqs.iloc[:, :start_idx[0]]
    for i in range(tmp_seqs.shape[0]):
        sequence = "".join(tmp_seqs.iloc[i, :][tmp_seqs.iloc[i, :] != "-"].values)
        if len(sequence) > 0:
            f.write(">"+str(tmp_seqs.index[i]))
            f.write("\n")
            f.write(sequence)
            f.write("\n")

# Supplemental script 4. Convert the output of `mclblastline` to Tribe data

The below code generate table of protein ids with tribe information ("Tribe_table.csv").

In [ ]:
#@title Upload result of mclblastline (like dump.out.XXX)
from google.colab import files
files = files.upload()
file_name = list(files.keys())[0]

In [ ]:
tribe_table = []
with open(file_name) as f:
    tribe=0
    for s_line in f:
        tribe+=1
        for pep_id in s_line.split("\t"):
            if pep_id.find("\n") > 0:
                tribe_table.append([f"tribe{tribe}", pep_id[:-1]])
            else:
                tribe_table.append([f"tribe{tribe}", pep_id])
pd.DataFrame(tribe_table).to_csv("Tribe_table.csv", index=None)